<a href="https://colab.research.google.com/github/aayushbhurtel/Data_Visualization/blob/main/Quiz15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [117]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re

In [118]:
r = requests.get('https://en.wikipedia.org/wiki/List_of_highest_mountains_on_Earth')
html_contents = r.text
html_soup = BeautifulSoup(html_contents,"lxml")
tables = html_soup.find_all('table',class_="wikitable")

df1 = pa.read_html(str(tables))[0]
df1.columns = df1.columns.droplevel(0).droplevel(0)
df1.head()

,Rank[dp 1],Mountain name(s),m,ft,m,ft,Range,Coordinates[dp 4],Parent mountain[dp 5],1st,y,n,Country (disputed claims in italics)
0,1,".mw-parser-output ul.cslist,.mw-parser-output ...",8848,"29,029[dp 7]",8848,29029,Mahalangur Himalaya,".mw-parser-output .geo-default,.mw-parser-outp...",—,1953,145,121,NepalChina
1,2,K2,8611,28251,4020,13190,Baltoro Karakoram,35°52′53″N 76°30′48″E﻿ / ﻿35.88139°N 76.51333°E,Mount Everest,1954,45,44,Pakistan[dp 8]China[12]
2,3,Kangchenjunga,8586,28169,3922,12867,Kangchenjunga Himalaya,27°42′12″N 88°08′51″E﻿ / ﻿27.70333°N 88.14750°E *,Mount Everest,1955,38,24,NepalIndia
3,4,Lhotse,8516,27940,610,2000,Mahalangur Himalaya,27°57′42″N 86°55′59″E﻿ / ﻿27.96167°N 86.93306°E,Mount Everest,1956,26,26,NepalChina
4,5,Makalu,8485,27838,2378,7802,Mahalangur Himalaya,27°53′23″N 87°05′20″E﻿ / ﻿27.88972°N 87.08889°E,Mount Everest,1955,45,—,NepalChina


as mentioned in the professor github, we can see issues in ft of first row. Here I am getting first row ft column.

In [119]:
df1.iloc[0,3]

'29,029[dp 7]'

In [120]:




re.sub(r"\D","",df1.iloc[0,3])



'290297'

here I am going to clean the headers of all the columns in the dataset

In [121]:
df1.columns

Index(['Rank[dp 1]', 'Mountain name(s)', 'm', 'ft', 'm', 'ft', 'Range',
       'Coordinates[dp 4]', 'Parent mountain[dp 5]', '1st', 'y', 'n',
       'Country (disputed claims in italics)'],
      dtype='object')

first, I am going to remove all the characters after the name like [dp 1] in Rank and more. I am using a lambda function to apply sub to all the headers.

In [122]:
header = df1.columns.map(lambda s: re.sub(r"\[(.+)\]","",s))
header

Index(['Rank', 'Mountain name(s)', 'm', 'ft', 'm', 'ft', 'Range',
       'Coordinates', 'Parent mountain', '1st', 'y', 'n',
       'Country (disputed claims in italics)'],
      dtype='object')

now, I am going to remove extra parenthesis from a second column header.

In [123]:
re.sub(r"\((.+)\)","",header[1])

'Mountain name'

here I am going to apply above removal to all of the header using lambda function.

In [124]:
header = header.map(lambda s: re.sub(r"\((.+)\)","",s))
header


Index(['Rank', 'Mountain name', 'm', 'ft', 'm', 'ft', 'Range', 'Coordinates',
       'Parent mountain', '1st', 'y', 'n', 'Country '],
      dtype='object')

using strip

In [125]:
header = header.map(lambda s: s.strip())
header

Index(['Rank', 'Mountain name', 'm', 'ft', 'm', 'ft', 'Range', 'Coordinates',
       'Parent mountain', '1st', 'y', 'n', 'Country'],
      dtype='object')

there should be no spaces in the header name so I am going to replace space with underscore on second header

In [126]:
re.sub(r" ","_", header[1])

'Mountain_name'

try and apply above code to all of the headers using lambda function

In [127]:
header = header.map(lambda s: re.sub(r" ","_",s))
header

Index(['Rank', 'Mountain_name', 'm', 'ft', 'm', 'ft', 'Range', 'Coordinates',
       'Parent_mountain', '1st', 'y', 'n', 'Country'],
      dtype='object')

now displaying all database

In [128]:
df1.columns = header

In [129]:
df1.head()

,Rank,Mountain_name,m,ft,m,ft,Range,Coordinates,Parent_mountain,1st,y,n,Country
0,1,".mw-parser-output ul.cslist,.mw-parser-output ...",8848,"29,029[dp 7]",8848,29029,Mahalangur Himalaya,".mw-parser-output .geo-default,.mw-parser-outp...",—,1953,145,121,NepalChina
1,2,K2,8611,28251,4020,13190,Baltoro Karakoram,35°52′53″N 76°30′48″E﻿ / ﻿35.88139°N 76.51333°E,Mount Everest,1954,45,44,Pakistan[dp 8]China[12]
2,3,Kangchenjunga,8586,28169,3922,12867,Kangchenjunga Himalaya,27°42′12″N 88°08′51″E﻿ / ﻿27.70333°N 88.14750°E *,Mount Everest,1955,38,24,NepalIndia
3,4,Lhotse,8516,27940,610,2000,Mahalangur Himalaya,27°57′42″N 86°55′59″E﻿ / ﻿27.96167°N 86.93306°E,Mount Everest,1956,26,26,NepalChina
4,5,Makalu,8485,27838,2378,7802,Mahalangur Himalaya,27°53′23″N 87°05′20″E﻿ / ﻿27.88972°N 87.08889°E,Mount Everest,1955,45,—,NepalChina


Header looks goooooooooood. onto next one.

# Describe last column and clean it

In [130]:
df2 = df1.iloc[:,-1]
df2

0                            NepalChina
1               Pakistan[dp 8]China[12]
2                            NepalIndia
3                            NepalChina
4                            NepalChina
                     ...               
115                               China
116                          NepalChina
117                  BhutanChina[dp 18]
118    IndiaChina[dp 10][dp 11]'[dp 12]
119                      Pakistan[dp 8]
Name: Country, Length: 120, dtype: object

using the techniques used above, I am goint ot remove all the extra expression other then letters in the column.

In [131]:
df2 = df2.apply(lambda x: re.sub(r"\[(.+)]","",x))
df2

0       NepalChina
1         Pakistan
2       NepalIndia
3       NepalChina
4       NepalChina
          ...     
115          China
116     NepalChina
117    BhutanChina
118     IndiaChina
119       Pakistan
Name: Country, Length: 120, dtype: object

here I am going to use a findall method of re library to separate the countries from camelCase word.

In [132]:
df2 = df2.apply(lambda x: re.findall(r"[A-Z][a-z]*",x))
df2

0       [Nepal, China]
1           [Pakistan]
2       [Nepal, India]
3       [Nepal, China]
4       [Nepal, China]
            ...       
115            [China]
116     [Nepal, China]
117    [Bhutan, China]
118     [India, China]
119         [Pakistan]
Name: Country, Length: 120, dtype: object

let us figure out the length of a row to see how many columns we can separate countries into using length function.

In [134]:
len(df1.columns)

13

In [135]:
df1.drop(df1.columns[12], axis = 1)

,Rank,Mountain_name,m,ft,m,ft,Range,Coordinates,Parent_mountain,1st,y,n
0,1,".mw-parser-output ul.cslist,.mw-parser-output ...",8848,"29,029[dp 7]",8848,29029,Mahalangur Himalaya,".mw-parser-output .geo-default,.mw-parser-outp...",—,1953,145,121
1,2,K2,8611,28251,4020,13190,Baltoro Karakoram,35°52′53″N 76°30′48″E﻿ / ﻿35.88139°N 76.51333°E,Mount Everest,1954,45,44
2,3,Kangchenjunga,8586,28169,3922,12867,Kangchenjunga Himalaya,27°42′12″N 88°08′51″E﻿ / ﻿27.70333°N 88.14750°E *,Mount Everest,1955,38,24
3,4,Lhotse,8516,27940,610,2000,Mahalangur Himalaya,27°57′42″N 86°55′59″E﻿ / ﻿27.96167°N 86.93306°E,Mount Everest,1956,26,26
4,5,Makalu,8485,27838,2378,7802,Mahalangur Himalaya,27°53′23″N 87°05′20″E﻿ / ﻿27.88972°N 87.08889°E,Mount Everest,1955,45,—
...,...,...,...,...,...,...,...,...,...,...,...,...
115,104,Noijin KangsangNorin Kang,7206,23642,2160,7090,Nagarze Himalaya,28°56′48″N 90°10′42″E﻿ / ﻿28.94667°N 90.17833°E,Gangkhar Puensum,1986,4,1
116,105,Langtang Ri,7205,23638,665,2182,Langtang Himalaya,28°22′53″N 85°41′01″E﻿ / ﻿28.38139°N 85.68361°E,Shishapangma,1981,4,0
117,106,Kangphu KangShimokangri,7204,23635,1244,4081,Lunana Himalaya,28°09′24″N 90°04′15″E﻿ / ﻿28.15667°N 90.07083°E,Tongshanjiabu,2002,1,0[20]
118,107,Singhi Kangri,7202,23629,730,2400,Siachen Karakoram,35°35′59″N 76°59′01″E﻿ / ﻿35.59972°N 76.98361°E,Teram Kangri III,1976,2,0


i am going to merge into the main dataframe.

In [136]:
df1 = pd.concat([df1,df2], axis = 1)
df1

,Rank,Mountain_name,m,ft,m,ft,Range,Coordinates,Parent_mountain,1st,y,n,Country,Country
0,1,".mw-parser-output ul.cslist,.mw-parser-output ...",8848,"29,029[dp 7]",8848,29029,Mahalangur Himalaya,".mw-parser-output .geo-default,.mw-parser-outp...",—,1953,145,121,NepalChina,"[Nepal, China]"
1,2,K2,8611,28251,4020,13190,Baltoro Karakoram,35°52′53″N 76°30′48″E﻿ / ﻿35.88139°N 76.51333°E,Mount Everest,1954,45,44,Pakistan[dp 8]China[12],[Pakistan]
2,3,Kangchenjunga,8586,28169,3922,12867,Kangchenjunga Himalaya,27°42′12″N 88°08′51″E﻿ / ﻿27.70333°N 88.14750°E *,Mount Everest,1955,38,24,NepalIndia,"[Nepal, India]"
3,4,Lhotse,8516,27940,610,2000,Mahalangur Himalaya,27°57′42″N 86°55′59″E﻿ / ﻿27.96167°N 86.93306°E,Mount Everest,1956,26,26,NepalChina,"[Nepal, China]"
4,5,Makalu,8485,27838,2378,7802,Mahalangur Himalaya,27°53′23″N 87°05′20″E﻿ / ﻿27.88972°N 87.08889°E,Mount Everest,1955,45,—,NepalChina,"[Nepal, China]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,104,Noijin KangsangNorin Kang,7206,23642,2160,7090,Nagarze Himalaya,28°56′48″N 90°10′42″E﻿ / ﻿28.94667°N 90.17833°E,Gangkhar Puensum,1986,4,1,China,[China]
116,105,Langtang Ri,7205,23638,665,2182,Langtang Himalaya,28°22′53″N 85°41′01″E﻿ / ﻿28.38139°N 85.68361°E,Shishapangma,1981,4,0,NepalChina,"[Nepal, China]"
117,106,Kangphu KangShimokangri,7204,23635,1244,4081,Lunana Himalaya,28°09′24″N 90°04′15″E﻿ / ﻿28.15667°N 90.07083°E,Tongshanjiabu,2002,1,0[20],BhutanChina[dp 18],"[Bhutan, China]"
118,107,Singhi Kangri,7202,23629,730,2400,Siachen Karakoram,35°35′59″N 76°59′01″E﻿ / ﻿35.59972°N 76.98361°E,Teram Kangri III,1976,2,0,IndiaChina[dp 10][dp 11]'[dp 12],"[India, China]"


In [137]:
df1.columns

Index(['Rank', 'Mountain_name', 'm', 'ft', 'm', 'ft', 'Range', 'Coordinates',
       'Parent_mountain', '1st', 'y', 'n', 'Country', 'Country'],
      dtype='object')

In [114]:
df1.iloc[:,-2]

0       [Nepal, China]
1           [Pakistan]
2       [Nepal, India]
3       [Nepal, China]
4       [Nepal, China]
            ...       
115            [China]
116     [Nepal, China]
117    [Bhutan, China]
118     [India, China]
119         [Pakistan]
Name: Country, Length: 120, dtype: object

In [138]:
df1

,Rank,Mountain_name,m,ft,m,ft,Range,Coordinates,Parent_mountain,1st,y,n,Country,Country
0,1,".mw-parser-output ul.cslist,.mw-parser-output ...",8848,"29,029[dp 7]",8848,29029,Mahalangur Himalaya,".mw-parser-output .geo-default,.mw-parser-outp...",—,1953,145,121,NepalChina,"[Nepal, China]"
1,2,K2,8611,28251,4020,13190,Baltoro Karakoram,35°52′53″N 76°30′48″E﻿ / ﻿35.88139°N 76.51333°E,Mount Everest,1954,45,44,Pakistan[dp 8]China[12],[Pakistan]
2,3,Kangchenjunga,8586,28169,3922,12867,Kangchenjunga Himalaya,27°42′12″N 88°08′51″E﻿ / ﻿27.70333°N 88.14750°E *,Mount Everest,1955,38,24,NepalIndia,"[Nepal, India]"
3,4,Lhotse,8516,27940,610,2000,Mahalangur Himalaya,27°57′42″N 86°55′59″E﻿ / ﻿27.96167°N 86.93306°E,Mount Everest,1956,26,26,NepalChina,"[Nepal, China]"
4,5,Makalu,8485,27838,2378,7802,Mahalangur Himalaya,27°53′23″N 87°05′20″E﻿ / ﻿27.88972°N 87.08889°E,Mount Everest,1955,45,—,NepalChina,"[Nepal, China]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,104,Noijin KangsangNorin Kang,7206,23642,2160,7090,Nagarze Himalaya,28°56′48″N 90°10′42″E﻿ / ﻿28.94667°N 90.17833°E,Gangkhar Puensum,1986,4,1,China,[China]
116,105,Langtang Ri,7205,23638,665,2182,Langtang Himalaya,28°22′53″N 85°41′01″E﻿ / ﻿28.38139°N 85.68361°E,Shishapangma,1981,4,0,NepalChina,"[Nepal, China]"
117,106,Kangphu KangShimokangri,7204,23635,1244,4081,Lunana Himalaya,28°09′24″N 90°04′15″E﻿ / ﻿28.15667°N 90.07083°E,Tongshanjiabu,2002,1,0[20],BhutanChina[dp 18],"[Bhutan, China]"
118,107,Singhi Kangri,7202,23629,730,2400,Siachen Karakoram,35°35′59″N 76°59′01″E﻿ / ﻿35.59972°N 76.98361°E,Teram Kangri III,1976,2,0,IndiaChina[dp 10][dp 11]'[dp 12],"[India, China]"
